# AgroPest-12 YOLOv8 baseline (simple)

A minimal YOLOv8 training and evaluation notebook using default settings (no custom augmentation or tuning).

In [ ]:
# Imports
from ultralytics import YOLO
import torch
from pathlib import Path
import yaml
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline

In [ ]:
# Device (GPU if available)
device = 0 if torch.cuda.is_available() else 'cpu'
print(f"Device: {torch.cuda.get_device_name(0) if device == 0 else 'CPU'}")

In [ ]:
# Dataset config (adjust path if you move this notebook)
data_yaml_path = Path('../dataset/data.yaml')
with open(data_yaml_path, 'r') as f:
    data = yaml.safe_load(f)
names = data.get('names', [])
print('Classes:', names)

In [ ]:
# Train (YOLOv8n default settings)
model = YOLO('yolov8n.pt')
results = model.train(
    data=str(data_yaml_path),
    epochs=50,
    imgsz=640,
    device=device,
    project='runs/detect',
    name='agropest_yolov8n_baseline'
)
print('Training done.')

In [ ]:
# Evaluate (validation) and show simple metrics and plots
val = model.val(data=str(data_yaml_path), split='val', device=device, imgsz=640, plots=True)
P = val.box.mp
R = val.box.mr
F1 = 2*P*R/(P+R)
print('Validation:')
print(' Precision:', P)
print(' Recall   :', R)
print(' F1       :', F1)
print(' mAP@0.5  :', val.box.map50)
print(' mAP@0.5:0.95:', val.box.map)

# Show confusion matrix and PR curve
cm = max(list(Path('runs').rglob('confusion_matrix.png')), key=os.path.getmtime)
pr = max(list(Path('runs').rglob('PR_curve.png')), key=os.path.getmtime)
for p in [cm, pr]:
    img = Image.open(p)
    plt.figure(figsize=(6,6))
    plt.axis('off')
    plt.imshow(img)
    plt.show()

In [ ]:
# Evaluate on test split (if present in data.yaml)
test = model.val(data=str(data_yaml_path), split='test', device=device, imgsz=640, plots=True)
P_t = test.box.mp
R_t = test.box.mr
F1_t = 2*P_t*R_t/(P_t+R_t)
print('Test:')
print(' Precision:', P_t)
print(' Recall   :', R_t)
print(' F1       :', F1_t)
print(' mAP@0.5  :', test.box.map50)
print(' mAP@0.5:0.95:', test.box.map)